In [1]:
import os
import time
from datetime import datetime

import pandas as pd
import pyodbc
import urllib

from azure.ai.anomalydetector import AnomalyDetectorClient
from azure.ai.anomalydetector.models import DetectionRequest, ModelInfo
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError

In [2]:
SUBSCRIPTION_KEY = os.environ["ANOMALY_DETECTOR_KEY"]
ANOMALY_DETECTOR_ENDPOINT = os.environ["ANOMALY_DETECTOR_ENDPOINT"]
TIME_SERIES_DATA_PATH = os.path.join("./sample_data", "request-data.csv")

# Prepare the data

In [3]:
def get_data_hack(sql):
    con=pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                           "Server={};"
                           "Database={};"
                           "uid={};"
                           "pwd={};")
    df=pd.read_sql(sql,con)
    con.close()
    return df

In [5]:
def saveSqlAsFiles(sql, FolderName = 'AzureBlobMinute'):
    df2 = get_data_hack(sql)
    df2['data'] = pd.to_datetime(df2.Dat).dt.tz_localize(None)
    df2 = df2.sort_values(by=['data'])
    df2.reset_index(inplace = True)
    
    for file in df2.TagName.unique():
        path = os.path.join("./"+FolderName, file.replace("/", "_") +".csv")
        tempdf = df2[df2.TagName == file]
        tempdf = tempdf[['Dat', 'MeanValue']]
        tempdf.rename(columns={'Dat':'timestamp','MeanValue':'value'}).to_csv(path, index=False)
    print("Data loaded into files")

In [39]:
def saveSqlAsFiles2(sql, file = 'Default', FolderName = 'AzureBlobOrgin'):
    df2 = get_data_hack(sql)
    df2['timestamp'] = df2.timestampCol.map(lambda x: datetime.strftime(x, '%Y-%m-%dT%H:%M:%SZ'))
    df2 = df2[['timestamp','value']]
    path = os.path.join("./"+FolderName, file.replace("/", "_") +".csv")
    df2.to_csv(path, index=False)
    print("Data loaded into files")

In [50]:
def saveDataFraneAsFiles(df2, FolderName = 'AzureBlobByDay'):
    df2['data'] = pd.to_datetime(df2.Dat).dt.tz_localize(None)
    df2 = df2.sort_values(by=['data'])
    df2.reset_index(inplace = True)
    
    for file in df2.TagName.unique():
        path = os.path.join("./"+FolderName, file.replace("/", "_") +".csv")
        tempdf = df2[df2.TagName == file]
        tempdf = tempdf[['Dat', 'MeanValue']]
        tempdf.rename(columns={'Dat':'timestamp','MeanValue':'value'}).to_csv(path, index=False)

## Data by Day

In [20]:
sql2 = '''
  select 
  cast(Year as varchar)+'-'+cast(Month as varchar)+'-'+cast(Day as varchar)+'T'+cast(Hour as varchar)+':'+'00:00Z' Dat, 
  [TagName], MeanValue from (
  select 
	 
	DATEPART(YEAR, [DateTime]) Year,
	case when DATEPART(MONTH, [DateTime]) < 10 
	then '0'+cast(DATEPART(MONTH, [DateTime]) as varchar)
	else cast(DATEPART(MONTH, [DateTime]) as varchar)
	end Month,
	case when DATEPART(Day, [DateTime]) < 10 
	then '0'+cast(DATEPART(Day, [DateTime]) as varchar)
	else cast(DATEPART(Day, [DateTime]) as varchar)
	end Day,
	case when DATEPART(Hour, [DateTime]) < 10 
	then '0'+cast(DATEPART(Hour, [DateTime]) as varchar)
	else cast(DATEPART(Hour, [DateTime]) as varchar)
	end Hour,
	--DATEPART(DAY, [DateTime]) Day,
	--DATEPART(HOUR, [DateTime]) Hour,
	[TagName],
	avg(Value) MeanValue
	
  from 
	  --(select top (100000) * from 
	  [Hack_Data]
	  --) t1
  group by  
	DATEPART(YEAR, [DateTime]),
	DATEPART(MONTH, [DateTime]),
	DATEPART(DAY, [DateTime]),
	DATEPART(HOUR, [DateTime]),
    
	[TagName] 
) t1
'''

In [21]:
%%time
df2 = get_data_hack(sql2)

Wall time: 1min 56s


In [22]:
df2

,Dat,TagName,MeanValue
0,2020-12-13T10:00:00Z,T1_podc-kolektorP,7.829622
1,2020-11-22T20:00:00Z,T1_Wapno_PrzeplywWody,5.327799
2,2020-12-14T16:00:00Z,T1_wydWaga6,2.136787
3,2020-12-16T03:00:00Z,T1_tempPieca,1191.240278
4,2020-11-17T21:00:00Z,ssawa2_T480t2_temp_w_loz_B_silnika,35.646389
...,...,...,...
508702,2020-02-18T12:00:00Z,T1_tempPieca,751.737705
508703,2020-11-26T02:00:00Z,ssawa1_T480t2_temp_w_lozB_silnika,61.726111
508704,2020-08-31T14:00:00Z,Ssawa2_Podcisnienie_przed_ssawa,12.188533
508705,2020-11-04T01:00:00Z,ssawa2_T559tD_temp_w_lozysku_D,52.562082


In [27]:
df2['data'] = pd.to_datetime(df2.Dat).dt.tz_localize(None)

In [30]:
df2 = df2.sort_values(by=['data'])

In [32]:
df2.reset_index(inplace = True)

In [45]:
%%time
for file in df2.TagName.unique():
    path = os.path.join("./AzureBlob", file.replace("/", "_") +".csv")
    tempdf = df2[df2.TagName == file]
    tempdf = tempdf[['Dat', 'MeanValue']]
    tempdf.rename(columns={'Dat':'timestamp','MeanValue':'value'}).to_csv(path, index=False)
    

Wall time: 16 s


## Data by Minute


In [9]:
sql1 = '''
  select 
  cast(Year as varchar)+'-'+cast(Month as varchar)+'-'+cast(Day as varchar)+'T'+cast(Hour as varchar)+':'+cast(Minute as varchar)+':00Z' Dat, 
  [TagName], MeanValue from (
  select 
	 
	DATEPART(YEAR, [DateTime]) Year,
	case when DATEPART(MONTH, [DateTime]) < 10 
	then '0'+cast(DATEPART(MONTH, [DateTime]) as varchar)
	else cast(DATEPART(MONTH, [DateTime]) as varchar)
	end Month,
	case when DATEPART(Day, [DateTime]) < 10 
	then '0'+cast(DATEPART(Day, [DateTime]) as varchar)
	else cast(DATEPART(Day, [DateTime]) as varchar)
	end Day,
	case when DATEPART(Hour, [DateTime]) < 10 
	then '0'+cast(DATEPART(Hour, [DateTime]) as varchar)
	else cast(DATEPART(Hour, [DateTime]) as varchar)
	end Hour,
    case when DATEPART(Minute, [DateTime]) < 10 
	then '0'+cast(DATEPART(Minute, [DateTime]) as varchar)
	else cast(DATEPART(Minute, [DateTime]) as varchar)
	end Minute,
	--DATEPART(DAY, [DateTime]) Day,
	--DATEPART(HOUR, [DateTime]) Hour,
	[TagName],
	avg(Value) MeanValue
	
  from 
	  --(select top (100000) * from 
	  [Hack_Data]
	  --) t1
  group by  
	DATEPART(YEAR, [DateTime]),
	DATEPART(MONTH, [DateTime]),
	DATEPART(DAY, [DateTime]),
	DATEPART(HOUR, [DateTime]),
    DATEPART(Minute, [DateTime]),
	[TagName] 
) t1
'''

In [7]:
%%time
df1 = get_data_hack(sql1)

OperationalError: ('08001', '[08001] [Microsoft][SQL Server Native Client 11.0]Named Pipes Provider: Could not open a connection to SQL Server [53].  (53) (SQLDriverConnect); [08001] [Microsoft][SQL Server Native Client 11.0]Login timeout expired (0); [08001] [Microsoft][SQL Server Native Client 11.0]A network-related or instance-specific error has occurred while establishing a connection to SQL Server. Server is not found or not accessible. Check if instance name is correct and if SQL Server is configured to allow remote connections. For more information see SQL Server Books Online. (53)')

In [10]:
%%time
saveSqlAsFiles(sql1)

Data loaded into files
Wall time: 1h 52min 19s


In [4]:
%%time
sql = """SELECT DISTINCT TagName from Hack_Data"""
df = get_data_hack(sql)
df

Wall time: 57.5 s


,TagName
0,HF1_Delta_P
1,HF1_MonitSpalin_HF1_NOx
2,HF1_MonitSpalin_HF1_O2
3,HF1_MonitSpalin_HF1_Przeplyw
4,HF1_MonitSpalin_HF1_Pyl
...,...
160,T1_wydWaga8
161,T1_wydWaga9
162,T1_ZB7_zapelnienie
163,T1_ZBBufor_Sortownia_zapelnienie


In [40]:
%%time
#a = ['T1_ZBPods_ciezar']
for tag in df.TagName:
    print(tag)
    sql2 = """
    SELECT 
    [DateTime]as timestampCol,
    [Value] as value
    from
    Hack_Data
    where TagName = '{}'
    order by DateTime
    """.format(tag)
    saveSqlAsFiles2(sql2, tag)
    

HF1_Delta_P
Data loaded into files
HF1_MonitSpalin_HF1_NOx
Data loaded into files
HF1_MonitSpalin_HF1_O2
Data loaded into files
HF1_MonitSpalin_HF1_Przeplyw
Data loaded into files
HF1_MonitSpalin_HF1_Pyl
Data loaded into files
HF1_MonitSpalin_HF1_SO2
Data loaded into files
HF1A_Delta_P
Data loaded into files
HF1A_MonitSpalin_HF1A_NOx
Data loaded into files
HF1A_MonitSpalin_HF1A_O2
Data loaded into files
HF1A_MonitSpalin_HF1A_Przeplyw
Data loaded into files
HF1A_MonitSpalin_HF1A_Pyl
Data loaded into files
HF1A_MonitSpalin_HF1A_SO2
Data loaded into files
Kalor_gaz_opal_piec_zapl
Data loaded into files
PT32_wydajnosc
Data loaded into files
PT35_wydajnosc
Data loaded into files
ssawa1_F561t_przep_ol_w_lozyskuD
Data loaded into files
ssawa1_F562t_przep_ol_w_lozyskuC
Data loaded into files
ssawa1_F563/1t_przep_ol_w_lozB_si
Data loaded into files
ssawa1_F563/2t_przep_ol_w_lozA_si
Data loaded into files
ssawa1_otwarcie_klapy
Data loaded into files
ssawa1_P521C_cis_oleju_lozyskC
Data loaded int

In [22]:
df3

,timestampCol,value
0,2020-01-01 00:00:10,3.538443
1,2020-01-01 00:00:20,3.504537
2,2020-01-01 00:00:30,3.470630
3,2020-01-01 00:00:40,3.436724
4,2020-01-01 00:00:50,3.412992
...,...,...
1588576,2020-12-17 22:47:20,3.935998
1588577,2020-12-17 22:47:30,3.915186
1588578,2020-12-17 22:47:40,3.894375
1588579,2020-12-17 22:47:50,3.873563


In [28]:
df3

,timestampCol,value,timestamp
0,2020-01-01 00:00:10,3.538443,2020-01-01 00:00:10
1,2020-01-01 00:00:20,3.504537,2020-01-01 00:00:20
2,2020-01-01 00:00:30,3.470630,2020-01-01 00:00:30
3,2020-01-01 00:00:40,3.436724,2020-01-01 00:00:40
4,2020-01-01 00:00:50,3.412992,2020-01-01 00:00:50
...,...,...,...
1588576,2020-12-17 22:47:20,3.935998,2020-12-17 22:47:20
1588577,2020-12-17 22:47:30,3.915186,2020-12-17 22:47:30
1588578,2020-12-17 22:47:40,3.894375,2020-12-17 22:47:40
1588579,2020-12-17 22:47:50,3.873563,2020-12-17 22:47:50


In [41]:
from os import listdir
from os.path import isfile, join
mypath = r"C:\Users\a0730244\Documents\AzureBlobByDay"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [49]:
emptyNameList = []
i = 0;

In [50]:
for file in onlyfiles:
    emptyNameList.append('series_'+str(i)+'.csv')
    i = i + 1

In [51]:
emptyNameList

['series_0.csv',
 'series_1.csv',
 'series_2.csv',
 'series_3.csv',
 'series_4.csv',
 'series_5.csv',
 'series_6.csv',
 'series_7.csv',
 'series_8.csv',
 'series_9.csv',
 'series_10.csv',
 'series_11.csv',
 'series_12.csv',
 'series_13.csv',
 'series_14.csv',
 'series_15.csv',
 'series_16.csv',
 'series_17.csv',
 'series_18.csv',
 'series_19.csv',
 'series_20.csv',
 'series_21.csv',
 'series_22.csv',
 'series_23.csv',
 'series_24.csv',
 'series_25.csv',
 'series_26.csv',
 'series_27.csv',
 'series_28.csv',
 'series_29.csv',
 'series_30.csv',
 'series_31.csv',
 'series_32.csv',
 'series_33.csv',
 'series_34.csv',
 'series_35.csv',
 'series_36.csv',
 'series_37.csv',
 'series_38.csv',
 'series_39.csv',
 'series_40.csv',
 'series_41.csv',
 'series_42.csv',
 'series_43.csv',
 'series_44.csv',
 'series_45.csv',
 'series_46.csv',
 'series_47.csv',
 'series_48.csv',
 'series_49.csv',
 'series_50.csv',
 'series_51.csv',
 'series_52.csv',
 'series_53.csv',
 'series_54.csv',
 'series_55.csv',
 '